1. Здійсніть імпорт необхідних пакетів.

In [147]:
!pip install category_encoders
!pip install gdown
import gdown
import warnings
import pandas as pd
import numpy as np
import category_encoders as ce
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_percentage_error
import category_encoders as ce
from scipy.stats import skew, kurtosis
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_percentage_error

2. Завантажте тренувальний mod_04_hw_train_data.csv і валідаційний mod_04_hw_valid_data.csv набори даних, доступні в репозиторії. Цільова змінна — рівень заробітної плати співробітників (Salary).

In [148]:
url = 'https://drive.google.com/uc?id=19Zv1ldUlHPUihmSdRmNhcPwer-AGLYyZ'
output = 'mod_04_hw_train_data.csv'
gdown.download(url, output, quiet=False)
data_train = pd.read_csv(output)

url_val = 'https://drive.google.com/uc?id=1NGRGiGJAoHau8GJMLPuDciQiGv4q2Y9a'
output_val = 'mod_04_hw_valid_data.csv'
gdown.download(url_val, output_val, quiet=False)
data_val = pd.read_csv(output_val)

Downloading...
From: https://drive.google.com/uc?id=19Zv1ldUlHPUihmSdRmNhcPwer-AGLYyZ
To: /Users/julia/Developer/GitDownload/Gits_DataScienceMasters/Machine_Learning_Masters/mod_04_hw_train_data.csv
100%|██████████| 17.2k/17.2k [00:00<00:00, 14.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1NGRGiGJAoHau8GJMLPuDciQiGv4q2Y9a
To: /Users/julia/Developer/GitDownload/Gits_DataScienceMasters/Machine_Learning_Masters/mod_04_hw_valid_data.csv
100%|██████████| 549/549 [00:00<00:00, 1.34MB/s]


In [158]:
print(data_train.head())
print(data_val.head(7))
print(data_train.shape)
print(data_val.shape)

                 Name  Phone_Number  Experience Qualification University  \
0  Jennifer Hernandez  120-602-1220         3.0           Msc      Tier2   
1      Timothy Walker  840-675-8650         5.0           PhD      Tier2   
2         David Duran  556-293-8643         5.0           Msc      Tier2   
3       Gloria Ortega  463-559-7474         3.0           Bsc      Tier3   
4      Matthew Steele  968-091-7683         5.0           Bsc      Tier2   

     Role Cert Date_Of_Birth  Salary  
0     Mid  Yes    25/08/1972   98000  
1  Senior  Yes    03/12/2013  135500  
2  Senior  Yes    19/07/2002  123500  
3     Mid   No    19/02/1970   85000  
4  Senior  Yes    20/02/1970  111500  
              Name  Phone_Number  Experience Qualification University    Role  \
0   Alvaro Johnson  320-636-8883           7           Bsc      Tier1  Senior   
1    Austin Powers  903-121-1691           2           Msc      Tier1     Mid   
2      Joshua Phil  673-972-2453           3           Bsc      Ti

3. Виконайте первинний дослідницький аналіз даних (EDA), визначте придатність і доцільність використання наявних в наборі ознак для моделювання.

In [150]:
numerical_features = ['Experience']
categorical_features = ['Qualification', 'Role', 'Cert']

In [151]:
columns_to_drop = ['Date_Of_Birth', 'University', 'Phone_Number', 'Name']

df_train = data_train.drop(columns=columns_to_drop)
df_val = data_val.drop(columns=columns_to_drop)

print("Remaining columns after dropping:")
print(df_train.columns)

print("Remaining  df_val columns after dropping:")
print(df_val.columns)

Remaining columns after dropping:
Index(['Experience', 'Qualification', 'Role', 'Cert', 'Salary'], dtype='object')
Remaining  df_val columns after dropping:
Index(['Experience', 'Qualification', 'Role', 'Cert', 'Salary'], dtype='object')


In [152]:
df_train = df_train.fillna(df_train.mode().iloc[0])
print(df_train.isnull().sum())

Experience       0
Qualification    0
Role             0
Cert             0
Salary           0
dtype: int64


In [153]:
numerical_features = ['Salary', 'Experience']

outlier_indices = []
z_score_threshold = 1.86

for feature in numerical_features:
    z_scores = stats.zscore(df_train[feature])
    feature_outliers = df_train.index[np.abs(z_scores) > z_score_threshold].tolist()
    outlier_indices.extend(feature_outliers)

outlier_indices = list(set(outlier_indices))

print("\nOutlier rows based on z-score:")
print(df_train.loc[outlier_indices])

df_cleaned = df_train.drop(index=outlier_indices).reset_index(drop=True)


Outlier rows based on z-score:
     Experience Qualification    Role Cert  Salary
75          1.0           Bsc  Junior   No   53500
236         1.0           Bsc  Junior   No   49500
203         1.0           Bsc  Junior   No   49500
239         1.0           Bsc  Junior   No   53500
176         1.0           Bsc  Junior   No   53500
148         1.0           Bsc  Junior   No   49500


4. Виконайте обробку числових ознак (трансформацію / нормалізацію за допомогою об’єктів StandardScaler або PowerTransformer з пакета sklearn) і кодування категоріальних (наприклад, за допомогою об’єктів OneHotEncoder / TargetEncoder з пакета sklearn або об’єктів з пакета category_encoders).

In [154]:
df_train = df_cleaned

numerical_features = ['Experience']
ordinal_features = ['Qualification', 'Role']
one_hot_features = ['Cert']

X_train = df_train.drop('Salary', axis=1).copy()
y_train = df_train['Salary']

X_train['Qualification'] = X_train['Qualification'].str.lower()
X_train['Role'] = X_train['Role'].str.lower()

X_train_numerical = X_train[numerical_features].values

qualification_mapping = {'bsc': 1, 'msc': 2, 'phd': 3}
role_mapping = {'junior': 1, 'mid': 2, 'senior': 3}

X_train['Qualification_Ordinal'] = X_train['Qualification'].map(qualification_mapping)
X_train['Role_Ordinal'] = X_train['Role'].map(role_mapping)

ordinal_feature_names = ['Qualification_Ordinal', 'Role_Ordinal']
X_train_ordinal = X_train[ordinal_feature_names].values

X_train['Cert'] = X_train['Cert'].str.lower()
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_train_one_hot = encoder.fit_transform(X_train[one_hot_features])

one_hot_feature_names = encoder.get_feature_names_out(one_hot_features)
all_feature_names = numerical_features + ordinal_feature_names + list(one_hot_feature_names)

X_train_transformed = np.hstack((X_train_numerical, X_train_ordinal, X_train_one_hot))

X_train_transformed_df = pd.DataFrame(X_train_transformed, columns=all_feature_names)
print(X_train_transformed_df.head())

   Experience  Qualification_Ordinal  Role_Ordinal  Cert_no  Cert_yes
0         3.0                    2.0           2.0      0.0       1.0
1         5.0                    3.0           3.0      0.0       1.0
2         5.0                    2.0           3.0      0.0       1.0
3         3.0                    1.0           2.0      1.0       0.0
4         5.0                    1.0           3.0      0.0       1.0


5. Побудуйте модель за допомогою об'єкта KNeighborsRegressor з пакета sklearn.

In [155]:
model = KNeighborsRegressor(n_neighbors=5, weights='distance',
                            algorithm='kd_tree', leaf_size=7,
                            p=2, metric='manhattan')

cv_scores = cross_val_score(
    model,
    X_train_transformed,
    y_train,
    cv=4,
    scoring='neg_mean_absolute_percentage_error'
)
model.fit(X_train_transformed, y_train)
cv_mape = -cv_scores
mean_cv_mape = cv_mape.mean()

print(f'Обчислена MAPE з крос-валідацією: {mean_cv_mape:.2%}')

Обчислена MAPE з крос-валідацією: 4.70%


6. Виконайте аналогічні етапи обробки і підготовки даних для валідаційного набору, подібно до того, як ми це робили у темі «Метод опорних векторів (SVM)» в розділі «Практика застосування SVM-класифікатора. Навчання й оцінка моделі. Приклад використання моделі)» .

In [156]:
X_valid = df_val.drop('Salary', axis=1)
y_valid = df_val['Salary']

X_valid_numerical = X_valid[numerical_features].values

for column in ['Qualification', 'Role', 'Cert']:
    df_val[column] = df_val[column].fillna(df_train[column].mode().iloc[0])

df_val['Qualification'] = df_val['Qualification'].str.lower()
df_val['Role'] = df_val['Role'].str.lower()
df_val['Cert'] = df_val['Cert'].str.lower()

X_val_numerical = df_val[numerical_features].values

df_val['Qualification_Ordinal'] = df_val['Qualification'].map(qualification_mapping)
df_val['Role_Ordinal'] = df_val['Role'].map(role_mapping)
X_val_ordinal = df_val[ordinal_feature_names].values

X_val_one_hot = encoder.transform(df_val[one_hot_features])

X_val_transformed = np.hstack((X_val_numerical, X_val_ordinal, X_val_one_hot))

7. Отримайте прогноз заробітної плати для нових об’єктів (працівників) із валідаційного набору, розрахуйте доцільні метрики точності регресійної моделі.

In [157]:
y_pred_valid = model.predict(X_val_transformed)

mape_valid = mean_absolute_percentage_error(y_valid, y_pred_valid)
print(f'MAPE for the validation set: {mape_valid:.2%}')

MAPE for the validation set: 10.39%


**Висновок**

У ході виконання цього проєкту було використано тренувальний набір даних розміром (249, 9) та валідаційний набір даних розміром (7, 9). Очевидна проблема, яка виникає при використанні таких малих вибірок, особливо валідаційної, полягає в недостатній кількості даних для достовірної оцінки моделі. Малий розмір валідаційного набору може призвести до високої варіативності метрик та ненадійних результатів, що ускладнює оцінку здатності моделі до узагальнення на нових даних.

Після проведення експлориторного аналізу даних (EDA) були отримані такі інсайти:

1. **Ordinalне кодування категоріальних змінних**: Змінні 'Qualification' та 'Role' мають природний порядок ('bsc' < 'msc' < 'phd' для 'Qualification' та 'junior' < 'mid' < 'senior' для 'Role'). Використання ordinalного кодування дозволило моделі врахувати цей порядок, що підвищило точність прогнозування.

2. **Особливості змінної 'Experience'**: Змінна 'Experience' містить дискретні значення в діапазоні від 1 до 5. Було вирішено не застосовувати до неї трансформацій або масштабування, оскільки це могло б спотворити її реальний вплив на заробітну плату. Використання її в початковому вигляді допомогло зберегти інформативність цієї змінної.

3. **Аналіз викидів**: При виявленні викидів за допомогою z-оцінки було використано поріг 1.86, який був обраний як оптимальний для моделі. Цей поріг дозволив ефективно ідентифікувати викиди без надмірного видалення даних, що сприяло покращенню якості навчання моделі та її здатності до узагальнення.

Пропуск кроку розділення даних за допомогою `train_test_split` може мати негативні наслідки для моделі. Без розподілу на тренувальну та тестову вибірки модель може бути піддана перенавчанню, оскільки вона "бачить" всі дані під час навчання. Це призводить до того, що модель може неадекватно реагувати на нові, невідомі дані, що знижує її практичну цінність.

Фінальна метрика моделі становила MAPE з крос-валідацією 4.70% та MAPE на валідаційному наборі 10.39%. Хоча показник на тренувальних даних є досить низьким, відносно високий показник на валідаційних даних може свідчити про перенавчання моделі або недостатню репрезентативність валідаційного набору через його малий розмір. Це ставить під сумнів достовірність отриманих результатів і вказує на необхідність збору більшого обсягу даних або перегляду методології моделювання.

У процесі роботи були виконані кроки, які не привели до бажаного результату та не були включені у фінальний код. Зокрема, використання різних енкодерів та трансформерів для категоріальних змінних не покращило продуктивність моделі. Також експерименти з масштабуванням числових змінних за допомогою скейлерів не дали значного ефекту, особливо враховуючи, що змінна 'Experience' має дискретний характер і не потребує масштабування.

Таким чином, для покращення моделі необхідно розглянути можливість збирання більшої кількості даних, переглянути методи передобробки та врахувати потенційні альтернативні алгоритми моделювання.